In [60]:
#%pip install requests_html

In [61]:
# change to store dataset
STORE_PUB_GS = True

# change for different location
# locations: "ravensburg", "mannheim", "heidenheim", "karlsruhe", "campus-horb", "stuttgart"

location = "ravensburg"

In [62]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from bs4 import BeautifulSoup
import re
import time
import datetime
import random

In [63]:
# get current year to save in applicable folder
current_year = datetime.date.today().year

# open session for html requests
SESSION = HTMLSession()

In [64]:
# get author names
pd_employees = pd.read_csv(f'../data/{current_year}/employees_{location}.csv')
author_names = pd_employees['employee_name_clean'].tolist()

In [65]:
def get_source(url):
    """Return the source code for the provided URL. 
    Args: 
        url (string): URL of the page to scrape.
    Returns:
        response (object): HTTP response object from requests_html. 
    """
    random_wait_time = random.randint(15,25)
    time.sleep(random_wait_time)

    try:
        response = SESSION.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [66]:
def get_author_id_query(soup):
    
    if soup.find("h4", class_ = "gs_rt2"): # check if is profile
        # get user ID for query
        query_with_id = soup.find("h4", class_ = "gs_rt2").find("a").get('href')
        
    else:
        query_with_id = None
        print(f"{author_name} does not have a profile on Google Scholar.")
        
    return query_with_id

In [67]:
def has_profile(soup):   
    if soup.find("h4", class_ = "gs_rt2"): # check if is profile
        return True
    else:
        return False

In [68]:
def remove_numbers(string_with_numbers):
    if not string_with_numbers:
        return None
    
    tokens = string_with_numbers.split(" ")
    cleaned_tokens = []
    for token in tokens:
        if not re.search("[0-9]", token):
            cleaned_tokens.append(token)
    return " ".join(cleaned_tokens) 

In [69]:
def get_all_publication_entries(soup):
    
    query_with_id = get_author_id_query(soup)
    soup = None

    if query_with_id != None:
        # get HTML response for author profile
        response = get_source(f"https://scholar.google.com{query_with_id}&cstart=0&pagesize=1001") # the last part is to get all entries

        if response:
            soup = BeautifulSoup(response.html.raw_html)

            #get article entries
            all_article_entries = soup.find_all("tr", class_ = "gsc_a_tr")
        
    else:
        all_article_entries = None
        
    return all_article_entries

In [70]:
def get_publication_info(entry):
    
    publication_title = ""
    publication_year = ""
    authors_list = []
    number_citations = ""
    
    # get authors and journal
    if entry.find("div", class_ = "gs_gray"):
        string_with_authors = entry.find("div", class_ = "gs_gray").string
        authors_list = string_with_authors.split(",")
        
        publisher_string = entry.find_all("div", class_ = "gs_gray")[1].text
        publisher = publisher_string.split(",")[0] #also has ISBN
        publisher = remove_numbers(publisher)

    # get title
    if entry.find("a", class_="gsc_a_at"):
        publication_title = entry.find("a", class_="gsc_a_at").string

    # get year
    if entry.find("span", class_ = "gsc_a_h gsc_a_hc gs_ibl"):
        publication_year = entry.find("span", class_ = "gsc_a_h gsc_a_hc gs_ibl").string

    # get number of citations
    if entry.find("a", class_ = "gsc_a_ac gs_ibl"):
        number_citations = entry.find("a", class_ = "gsc_a_ac gs_ibl").string
        
    
    
    return publication_title, publication_year, authors_list, number_citations, publisher

In [71]:
def get_dataframe_with_publications(soup, author_name):
    
    publication_titles = []
    publication_years = []
    publishers = []
    authors_lists = []
    number_citations_list = []
    author_name_list = []
    
    all_publication_entries = get_all_publication_entries(soup)
    
    if all_publication_entries:
        
        for publication_entry in all_publication_entries:
            publication_title, publication_year, authors_list, number_citations, publisher = get_publication_info(publication_entry)

            publication_titles.append(publication_title)
            publication_years.append(publication_year)
            publishers.append(publisher)
            authors_lists.append(authors_list)
            number_citations_list.append(number_citations)
            author_name_list.append(author_name)


    data = {'PUB_TITLE': publication_titles, 
            'PUB_YEAR': publication_years,
            'PUB_PUBLISHER': publishers,
            'PUB_AUTHORS': authors_lists, 
            'PUB_CITATIONS': number_citations_list, 
            'AUTHOR_NAME': author_name_list}
    
    return pd.DataFrame(data)

In [72]:
def get_dataframe_with_publications_no_profile(soup, author_name):

    publication_titles = []
    publication_years = []
    publishers = []
    authors_lists = []
    number_citations_list = []
    author_name_list = []
    
    number_publications = 1

    search_result = soup.find("div", id="gs_ab_md").find("div", class_="gs_ab_mdw").text
    
    if re.search(" [0-9]* result", search_result):
        number_publications = int(re.search(" [0-9]* result", search_result).group().strip().split(" ")[0])
        
    # first page
    
    for entry in soup.find_all("div", class_="gs_ri"):

        # and journal
        if entry.find("div", class_="gs_a"):
            # get authors
            authors_list = entry.find("div", class_="gs_a").text.split("-")[0].replace("\xa0", "").split(",")

            # get year
            if re.search("[0-9]{4}", entry.find("div", class_="gs_a").text):
                publication_year = re.search("[0-9]{4}", entry.find("div", class_="gs_a").text).group()
            else:
                publication_year = None

            # get publisher
            journal_string = entry.find("div", class_="gs_a").text.split("-")[1]
            if "," in journal_string:
                publisher = journal_string.split(",")[0]
            else:
                publisher = None
                
            publisher = remove_numbers(publisher)

        # get title
        if entry.find("h3", class_="gs_rt"):
            publication_title = entry.find("h3", class_="gs_rt").text.replace("[PDF]", "").strip()

        # get number of citations
        footer_elements = entry.find("div", class_="gs_fl").find_all("a")
        number_citations = None

        for element in footer_elements:
            if "Cited by" in element.text:
                number_citations = int(re.search("[0-9]*$", element.text).group())


        publication_titles.append(publication_title)
        publication_years.append(publication_year)
        publishers.append(publisher)
        authors_lists.append(authors_list)
        number_citations_list.append(number_citations)
        author_name_list.append(author_name)

        

    # all other pages
    for current_page in range(10, number_publications, 10):
        next_page = get_source(f"https://scholar.google.cz/scholar?start={current_page}&hl=en&as_sdt=0%2C5&as_vis=1&q=author%3A%22{author_name_string}%22&btnG=")
        soup = BeautifulSoup(next_page.html.raw_html)

        for entry in soup.find_all("div", class_="gs_ri"):

            # and journal
            if entry.find("div", class_="gs_a"):
                # get authors
                authors_list = entry.find("div", class_="gs_a").text.split("-")[0].replace("\xa0", "").split(",")

                # get year
                if re.search("[0-9]{4}", entry.find("div", class_="gs_a").text):
                    publication_year = re.search("[0-9]{4}", entry.find("div", class_="gs_a").text).group()
                else:
                    publication_year = None

                # get publisher
                journal_string = entry.find("div", class_="gs_a").text.split("-")[1]
                if "," in journal_string:
                    publisher = journal_string.split(",")[0]
                else:
                    publisher = None
                    
                publisher = remove_numbers(publisher)

            # get title
            if entry.find("h3", class_="gs_rt"):
                publication_title = entry.find("h3", class_="gs_rt").text.replace("[PDF]", "").strip()

            # get number of citations
            footer_elements = entry.find("div", class_="gs_fl").find_all("a")
            number_citations = None
            
            for element in footer_elements:
                if "Cited by" in element.text:
                    number_citations = int(re.search("[0-9]*$", element.text).group())


            publication_titles.append(publication_title)
            publication_years.append(publication_year)
            publishers.append(publisher)
            authors_lists.append(authors_list)
            number_citations_list.append(number_citations)
            author_name_list.append(author_name)

    data = {'PUB_TITLE': publication_titles, 
            'PUB_YEAR': publication_years,
            'PUB_PUBLISHER': publishers,
            'PUB_AUTHORS': authors_lists, 
            'PUB_CITATIONS': number_citations_list, 
            'AUTHOR_NAME': author_name_list}

    return pd.DataFrame(data)

In [73]:
# loop through authors and collect publications in dataframe

pd_result = pd.DataFrame(columns=['AUTHOR_NAME', 'PUB_TITLE', 'PUB_YEAR', 'PUB_AUTHORS', 'PUB_PUBLISHER', 'PUB_CITATIONS'])

for author_name in author_names:
    
    # get HTML response for author
    author_name_string = author_name.replace(" ", "+")
    response = get_source(f"https://scholar.google.cz/scholar?hl=en&as_sdt=0%2C5&as_vis=1&q=author%3A%22{author_name_string}%22&btnG=")
    soup = BeautifulSoup(response.html.raw_html)
    
    if has_profile(soup):
        pd_publications = get_dataframe_with_publications(soup, author_name)

    else:
        pd_publications = get_dataframe_with_publications_no_profile(soup, author_name)
        
    if pd_publications.empty:
        print(f"No publications found for {author_name}.")
    
    else:
        pd_result = pd_result.append(pd_publications, ignore_index=True, sort=False)

pd_result = pd_result.reset_index(drop=True)

No publications found for Franziska Baar.
No publications found for Jürgen Brath.
No publications found for Bhagyaraj Dharmana.
No publications found for Vera Engelbart.
No publications found for Patrick Gieger.
No publications found for Chaitanya Grandhi.
No publications found for Mathias Hassenstein.
No publications found for Rajesh Kallur Krishnamoorthy.
No publications found for John-Dean Kasher.
No publications found for Christina Krumm.
No publications found for Albrecht Linkohr.
No publications found for Holger Lund.
No publications found for Stefan Luppold.
No publications found for Thomas Mannchen.
No publications found for Conny Mayer-Bonde.
No publications found for Christoph Moser.
No publications found for Herbert Moser.
No publications found for Bodo Möslein-Tröppner.
No publications found for Maren Müller.
No publications found for Christoph Neef.
No publications found for Thomas Nickel.
No publications found for Uwe Nölte.
No publications found for Marc Nutzmann.
No pub

In [74]:
if STORE_PUB_GS:
    pd_result.to_csv(f'../data/{current_year}/publications_{location}_gs.csv', index=False)

In [75]:
pd_result

,AUTHOR_NAME,PUB_TITLE,PUB_YEAR,PUB_AUTHORS,PUB_PUBLISHER,PUB_CITATIONS
0,Atheer Al-Tameemi,Estimation of planetary surface ages using ima...,2018,[A Al],None,NaN
1,Thomas Asche,Validation of the COMPASS force field for comp...,2017,"[TS Asche, P Behrens, AM Schneider]",None,14
2,Thomas Asche,[BOOK][B] Das Sicherheitsverhalten von Konsume...,1990,[T Asche ],None,26
3,Thomas Asche,Die Ergebnisse der empirischen Analyse zum Sic...,1990,[T Asche],Das Sicherheitsverhalten von Konsumenten,NaN
4,Thomas Asche,Die Datengewinnung zur Analyse des Sicherheits...,1990,[T Asche],Das Sicherheitsverhalten von Konsumenten,NaN
...,...,...,...,...,...,...
1052,Frank Lehmann,""" Last-Mile"" preparation for a potential disas...",2009,"[H Taubenböck, N Goseberg, N Setiadi…]",… Hazards and Earth …,151
1053,Frank Lehmann,Influence of surface conditioning and cleaning...,2011,"[A Attia, F Lehmann, M Kern]",Dental Materials,148
1054,Frank Lehmann,Acoustic emission analysis for the quantificat...,2012,"[K Van Tittelboom, N De Belie, F Lehmann…]",… and Building Materials,142
1055,Frank Lehmann,[HTML][HTML] Interleukin-33-activated islet-re...,2017,"[E Dalmas, FM Lehmann, E Dror, S Wueest, C...",Immunity,85
